In [1]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
spacy.require_gpu()
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup

from datetime import date

c:\Users\hydro\anaconda3\envs\cov\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
data_path = r"D:\covid research\output\step 2 output\2020-06-es"
days=os.listdir(data_path)
# days =[ i for i in days if i.split('.')[0].split('-')[-1]=='es']
print(days)

data_list = []
for i in days:
    file_path  = i
    path = os.path.join(data_path, file_path)
    print(path)
    df_day = pd.read_csv(path,lineterminator='\n')
    data_list.append(df_day)
df = pd.concat(data_list)

['step2-output-ES-2020-06-01.csv', 'step2-output-ES-2020-06-02.csv', 'step2-output-ES-2020-06-04.csv', 'step2-output-ES-2020-06-05.csv', 'step2-output-ES-2020-06-06.csv', 'step2-output-ES-2020-06-07.csv', 'step2-output-ES-2020-06-08.csv', 'step2-output-ES-2020-06-09.csv', 'step2-output-ES-2020-06-10.csv', 'step2-output-ES-2020-06-11.csv', 'step2-output-ES-2020-06-12.csv', 'step2-output-ES-2020-06-13.csv', 'step2-output-ES-2020-06-14.csv', 'step2-output-ES-2020-06-15.csv', 'step2-output-ES-2020-06-16.csv', 'step2-output-ES-2020-06-17.csv', 'step2-output-ES-2020-06-18.csv', 'step2-output-ES-2020-06-19.csv', 'step2-output-ES-2020-06-20.csv', 'step2-output-ES-2020-06-21.csv', 'step2-output-ES-2020-06-22.csv', 'step2-output-ES-2020-06-23.csv', 'step2-output-ES-2020-06-24.csv', 'step2-output-ES-2020-06-25.csv', 'step2-output-ES-2020-06-26.csv', 'step2-output-ES-2020-06-27.csv', 'step2-output-ES-2020-06-28.csv', 'step2-output-ES-2020-06-29.csv', 'step2-output-ES-2020-06-30.csv']
D:\covid rese

In [3]:
df_date = df.copy(deep=True)
df_date['day'] = df_date['created_at'].apply(lambda x: x.split(' ')[0])
df_date['hour'] = df_date['created_at'].apply(lambda x: x.split(' ')[1])
df_date['hour'] = df_date['hour'].apply(lambda x: x.split('-')[0])
#start_date = '2020-03-' + '%.2d' % start
#end_date = '2020-03-' + '%.2d' % (end + 1)

#selected_date = (df_date['day'] != start_date) & (df_date['day'] != end_date)
df_01 = df_date.copy(deep=True)
df_01 = df_01.reset_index(drop=True)
day_count = df_01.groupby('day').count()['text']


# Preprocessing

In [4]:
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
df_01['text'] = df_01['text'].apply(lambda x: remove_emoji(x))
df_01['text'] = df_01['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_01['text'] = df_01['text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
df_01['text'] = df_01['text'].apply(lambda x: re.sub('\[[^]]*\]', '', x))
df_01['text'] = df_01['text'].apply(lambda x: re.sub(r'http\S+', '', x))
df_01['text'] = df_01['text'].str.replace('[^\w\s]','')
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('spanish'))
punctuation = list(string.punctuation)
stop.update(punctuation)
df_01['text'] = df_01['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
def remove_xml(text):
    return re.sub(r'<[^<]+?>', '', text)

def remove_newlines(text):
    return text.replace('\n', ' ') 
    

def remove_manyspaces(text):
    return re.sub(r'\s+', ' ', text)

def clean_text(text):
    text = remove_xml(text)
    text = remove_newlines(text)
    text = remove_manyspaces(text)
    return text
df_01['text']= df_01['text'].apply(clean_text)
df_02 = df_01.copy()
nlp = spacy.load('es_core_news_sm')
# lemmatizatioin using Spacy to count the appearance of each word
def space (comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
df_02['text']= df_02['text'].apply(space)
df_02['text'].head(20) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0     rt andyws0k coronavirus malén galmarini decir ...
1     rt soulsearcher_dr sanitario covid_19 improvis...
2     rt elcoya1977 buen día ginés garcía armar plen...
3     rt mazzolenijulio informe covid19 1 junio proc...
4     rt droscarnoguerar atención seguir aumento tas...
5     rt mifutbolecuador experto italiano asegurar c...
6     rt gabrielrufiar pandemia surgido chino pandem...
7     rt m4neugly coronavirus ver centro atención mundo
8     pedianu putin soltar león call obligar gente q...
9     covid19 venir próximo semana conversar exdirec...
10    rt carlitoselnicar conmovedor pintor jerezano ...
11    rt latercerar catterina ferreccio epidemiólogo...
12    rt gabrielrufiar pandemia surgido chino pandem...
13    rt lolangostinir dónde creéis ir instagram dig...
14    rt pabloagustin pelicula suspenso drama acción...
15    rt m4neugly coronavirus ver centro atención mundo
16    rt jesuscintoro fortunir 23 español rico crece...
17    rt iamv2020 fin mundo ebola coronavirus pu

# NRC

In [6]:
ncr = pd.read_excel("D:/covid research/src/local_exe/step 3/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx")
ncr = ncr[['English (en)', 'Spanish (es)', 'Positive', 'Negative', 'Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']]
ncr = ncr[ncr['Spanish (es)'] != 'NO TRANSLATION'].reset_index(drop=True)
emotions = ['Anger', 'Anticipation','Disgust','Fear', 'Joy','Sadness', 'Surprise', 'Trust']
def join_edited_string(edited_tweets):
    
    edited_string = ''
    for row in edited_tweets:
        edited_string = edited_string + ' ' + row
        
    return edited_string

joined_string = join_edited_string(df_01['text'])

tokens = joined_string.split(' ')

joined_string1 = join_edited_string(df_02['text'])
tokens1 = joined_string1.split(' ')

unique_words = set(tokens)
unique_words1 = set(tokens1)
unique_words.update(unique_words1)
word_to_ind = dict((word, i) for i, word in enumerate(unique_words))
ind_to_word = dict((i, word) for i, word in enumerate(unique_words))

In [7]:
import nltk
nltk.download('punkt')
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hydro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df_tweets = df_01.copy(deep=True)
df_tweets1 = df_02.copy(deep=True)
emo_info = {'emotion':'' , 'emo_frq': defaultdict(int) }
list_emotion_counts = []
for emotion in emotions:
        emo_info = {}
        emo_info['emotion'] = emotion
        emo_info['emo_frq'] = defaultdict(int)
        list_emotion_counts.append(emo_info)
nrow = df_tweets.shape[0]
df_emotions = pd.DataFrame(0, index=df_tweets.index, columns=emotions)

In [9]:
process_count=0
for i in range(nrow):
        process_count+=1
        if process_count %100==0:
            print(process_count)
            print(df_tweets.loc[i])
        tweet = word_tokenize(df_tweets.loc[i]['text'], language='spanish')
        for word in tweet:
            word_stemmed = word
            result = ncr[ncr['Spanish (es)'] == word_stemmed]
            if not result.empty:
                # update the tweet-emotions counts
                for idx, emotion in enumerate(emotions):
                    df_emotions.at[i, emotion] += result[emotion].iloc[0]
                    # update the frequencies dictionary list
                    if result[emotion].iloc[0].any():
                        try:
                            list_emotion_counts[idx]['emo_frq'][word_to_ind[word]] += 1
                        except:
                            continue
            else:
                doc = nlp(word)
                word_stemmed1 = " ".join([token.lemma_ for token in doc]).lower()
                result1 = ncr[ncr['Spanish (es)'] == word_stemmed1]
                if not result1.empty:
                    # update the tweet-emotions counts
                    for idx, emotion in enumerate(emotions):
                        df_emotions.at[i, emotion] += result1[emotion].iloc[0]
                        # update the frequencies dictionary list
                        if result1[emotion].iloc[0].any():
                            try:
                                list_emotion_counts[idx]['emo_frq'][word_to_ind[word_stemmed1]] += 1
                            except:
                                continue

100
created_at                                      2020-06-02 00-52-44
id_str                                          1267620129019396098
text              último indígenas colombia gracias manuelandrio...
user              {'id': 114511956, 'id_str': '114511956', 'name...
lang                                                             es
favorite_count                                                  0.0
retweet_count                                                   0.0
quote_count                                                     0.0
location                                                   Colombia
country\r                                                colombia\r
day                                                      2020-06-02
hour                                                             00
Name: 99, dtype: object
200
created_at                                      2020-06-01 06-44-00
id_str                                          1267346140313288705
text            

In [10]:
df_tweets = pd.concat([df_tweets, df_emotions], axis=1)

In [11]:
df_tweets.to_csv(r"D:\covid research\output\step 3 output\2020-06-ES.csv")